In [4]:
import pyopencl as cl
import numpy as np
import time

# Set up OpenCL context and command queue
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

# Set array size and create random data for input arrays
array_size = 10**7
a_np = np.random.rand(array_size).astype(np.float32)
b_np = np.random.rand(array_size).astype(np.float32)

# Create OpenCL buffers for input and output arrays
a_g = cl.Buffer(ctx, cl.mem_flags.READ_ONLY, a_np.nbytes)
b_g = cl.Buffer(ctx, cl.mem_flags.READ_ONLY, b_np.nbytes)
c_g = cl.Buffer(ctx, cl.mem_flags.WRITE_ONLY, a_np.nbytes)

# Copy data to the device (GPU) memory
cl.enqueue_copy(queue, a_g, a_np)
cl.enqueue_copy(queue, b_g, b_np)

# Define OpenCL kernel code for element-wise vector addition
kernel_code = """
__kernel void vector_add(__global const float *a,
                          __global const float *b,
                          __global float *c)
{
    int gid = get_global_id(0);
    c[gid] = a[gid] + b[gid];
}
"""

# Compile kernel code
prg = cl.Program(ctx, kernel_code).build()

# Set up and run kernel
global_size = (array_size,)
local_size = None
kernel_args = (a_g, b_g, c_g)
t1 = time.time()
event = prg.vector_add(queue, global_size, local_size, *kernel_args)
event.wait()
t2 = time.time()

# Copy result from GPU memory back to host memory
c_np = np.empty_like(a_np)
cl.enqueue_copy(queue, c_np, c_g)

# Check the result and compute the error
error = np.abs(c_np - (a_np + b_np)).max()
print("Error: {}".format(error))

# Print the execution time
print("Execution time: {:.5f} seconds".format(t2 - t1))


Error: 0.0
Execution time: 0.00180 seconds


/home/mahir/.local/lib/python3.11/site-packages/pyopencl/__init__.py:268: CompilerWarning: From-source build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'gfx1032' on 'AMD Accelerated Parallel Processing' at 0x5598d2d96380> succeeded, but said:

1 warning generated.

  warn(text, CompilerWarning)


In [5]:
import pyopencl as cl


In [6]:
platforms = cl.get_platforms()


In [7]:
for platform in platforms:
    devices = platform.get_devices()
    for device in devices:
        print("Platform: {}, Device: {}".format(platform.name, device.name))


Platform: AMD Accelerated Parallel Processing, Device: gfx1032
